In [37]:
import requests
import json
import base64
import urllib.parse

from requests import post, get

In [38]:
client_id = "5d38bc500c7541c2a2a407af5adb26a8"
client_secret = "f6c6b5e586af4ed4ae6ab479f75d9d7e"

In [39]:
def get_token():
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {"Authorization": "Basic " + auth_base64,
               "Content-Type": "application/x-www-form-urlencoded"}
    data = {"grant_type":"client_credentials"}

    result = post(url, headers = headers, data = data)
    json_result = json.loads(result.content)
    token = json_result["access_token"]
    
    return token

In [40]:
def get_auth_header(token):
    return {"Authorization": "Bearer " + token}

In [41]:
def search_for_artist(token, artist_name):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"?q={artist_name}&type=artist&limit=1"

    query_url = url + query
    result = get(query_url, headers = headers)
    json_result = json.loads(result.content)["artists"]["items"]

    if len(json_result) == 0:
        print("No artist with this name exists")
        return None
        
    return json_result[0]

In [42]:
def get_albums_by_artist(token, artist_id):
    url = f"https://api.spotify.com/v1/artists/{artist_id}/albums"
    headers = get_auth_header(token)
    
    result = get(url, headers = headers)
    json_result = json.loads(result.content)

    return json_result

In [43]:
def get_tracks_by_album(token, album_id):
    url = f"https://api.spotify.com/v1/albums/{album_id}/tracks"
    headers = get_auth_header(token)
    
    result = get(url, headers = headers)
    json_result = json.loads(result.content)
    
    return json_result

In [55]:
# Get all of Paul McCartney's music data
token = get_token()

# Create new file that will be written later. 
filename = "paul_john_album_songs.csv"

# Open file 
my_file = open(filename,"w", encoding="utf-8")

# Write column names in first row
write_this = "Artist, Album, Track \n"
my_file.write(write_this)
my_file.close()

artist = search_for_artist(token, "Paul McCartney")
artist_id = artist["id"]
albums = get_albums_by_artist(token, artist_id)

# Open file for appending to
my_file = open(filename, "a", encoding="utf-8")

for i in range(len(albums['items'])):
    # Define artist, this is our label
    artist = "Paul McCartney"

    # Define album name
    album = albums['items'][i]['name']

    # Get tracks from album
    album_id = albums['items'][i]['id']
    tracks = get_tracks_by_album(token, album_id)

    for j in range(len(tracks['items'])):
        track = tracks['items'][j]['name'] 

        write_this = str(artist) + "," + str(album) + "," + str(track) + "\n"
        my_file.write(write_this)
    
my_file.close()

# Now John Lennon
artist = search_for_artist(token, "John Lennon")
artist_id = artist["id"]
albums = get_albums_by_artist(token, artist_id)

# Open file for appending to
my_file = open(filename, "a", encoding="utf-8")

for i in range(len(albums['items'])):
    # Define artist, this is our label
    artist = "John Lennon"

    # Define album name
    album = albums['items'][i]['name']

    # Get tracks from album
    album_id = albums['items'][i]['id']
    tracks = get_tracks_by_album(token, album_id)

    for j in range(len(tracks['items'])):
        track = tracks['items'][j]['name'] 

        write_this = str(artist) + "," + str(album) + "," + str(track) + "\n"
        my_file.write(write_this)
    
my_file.close()